In [1]:
# CONFIG CELL
from notebook_utils import set_root_directory

set_root_directory()

In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd

from app import constants
from app.imputers import NearestSensorImputer

In [3]:
ANALYZED_VARIABLE = constants.PM10

SENSOR_METADATA = "input_files/sensor_metadata.parquet"
MEASUREMENTS_24H = "input_files/measurements_24h.parquet"

OUTPUT_FILE_NEAREST = f"input_files/nearest_imputed_measurements_24h_{ANALYZED_VARIABLE}.parquet"
OUTPUT_FILE_LAST = f"input_files/last_imputed_measurements_24h_{ANALYZED_VARIABLE}.parquet"
OUTPUT_FILE_COMBINED = f"input_files/combined_imputed_measurements_24h_{ANALYZED_VARIABLE}.parquet"

MIN_YEAR = 2017
MAX_YEAR = 2023

In [4]:
sensor_metadata = pd.read_parquet(SENSOR_METADATA)
raw_measurements = pd.read_parquet(MEASUREMENTS_24H)

In [5]:
measurements = raw_measurements[[constants.TIMESTAMP_COLUMN, constants.UNIQUE_ID, ANALYZED_VARIABLE]]
measurements = measurements.query(
    f"{constants.TIMESTAMP_COLUMN}.dt.year >= {MIN_YEAR} and "
    f"{constants.TIMESTAMP_COLUMN}.dt.year <= {MAX_YEAR}"
)
measurements.shape

(2267172, 3)

In [6]:
loc_sensor = sensor_metadata[[constants.SENSOR_ID, constants.LATITUDE, constants.LONGITUDE]]
gdf_loc_sensor = gpd.GeoDataFrame(
    loc_sensor,
    geometry=gpd.points_from_xy(loc_sensor[constants.LONGITUDE], loc_sensor[constants.LATITUDE]),
    crs=constants.GLOBAL_EPSG,
)
gdf_loc_sensor = gdf_loc_sensor.to_crs(constants.POLAND_EPSG)
distance_matrix = gdf_loc_sensor.geometry.apply(
    lambda x: gdf_loc_sensor.distance(x)
).values.tolist()

c:\Users\Mambo\.conda\envs\ispies\Lib\site-packages\shapely\measurement.py:81: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


In [7]:
nearest_sensor_imputer = NearestSensorImputer(
    distance_matrix=distance_matrix,
    sensor_ids=gdf_loc_sensor[constants.SENSOR_ID].values.tolist()
)
nearest_sensor_imputed_measurements = nearest_sensor_imputer.fit_transform(
    X=measurements
)
nearest_sensor_imputed_measurements.to_parquet(OUTPUT_FILE_NEAREST, index=False)

[########################################] | 100% Completed | 187.83 s
